In [8]:
from z3 import *
import math

## Utils

#### At most/least one & exactly one

In [9]:
def at_least_one_seq(bool_vars):
    return Or(bool_vars)

def at_most_one_seq(x, name):
    # bool_vars renamed to x for simplicity
    n = len(x)
    if n == 1:
        return True
    s = [Bool(f"s_{i}_{name}") for i in range(n-1)]     # "name" is used in order to create a
                                                        # unique set of variables s for each constraint
                                                        # of the kind at_most_one_seq, so that they do not overlap 
                                                        # for different constraints!
                                                        # N.B. len(s) = n-1 != n = len (x)
    clauses = []
    clauses.append(Or(Not(x[0]), s[0]))                 # x[0] -> s[0] (s[i] modeled as: s[i] is true if the SUM UP TO index i IS 1!) i.e. x_1 -> s_1 in math notation
    for i in range(1, n-1):
        clauses.append(Or(Not(x[i]), s[i]))             # these two clauses model (x[i] v s[i-1]) -> s[i]
        clauses.append(Or(Not(s[i-1]), s[i]))
        clauses.append(Or(Not(s[i-1]), Not(x[i])))      # this one models s[i-1] -> not x[i]
    clauses.append(Or(Not(s[-1]), Not(x[-1])))          # s[n-2] -> not x[n-1]  i.e. s_(n-1) -> s_n in the mathematical notation (1-based)
    return And(clauses)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

#### Conversion from int to bin

In [10]:
def find_max_digits(s, l): # TODO Idea: optimize number of digits depending on cases
    return math.ceil(math.log2(max(sum(s), max(l))))

In [11]:
def int_to_bin(x, digits=5):  # TODO read: #https://ericpony.github.io/z3py-tutorial/guide-examples.htm
    x_bin = [(x%(2**(i+1)) // 2**i)==1 for i in range(digits-1,-1,-1)]
    return x_bin

#### Operations on binary numbers

In [12]:
def equal(v, u, digits):
    return And([v[k]==u[k] for k in range(digits)])

def all_false(v):
    return And([Not(v[k]) for k in range(len(v))])

def leq(v, u, digits): #v<=u
    if digits == 1:
        return Or(v[0]==u[0], And(Not(v[0]), u[0]))
    else:
        return Or(And(Not(v[0]), u[0]), 
                  And(v[0]==u[0], leq(v[1:], u[1:], digits-1)))


In [13]:
def sum_bin(a_bin, b_bin, d_bin, digits, name):
    """Encodes into a SAT formula the binary sum {a_bin + b_bin = d_bin}, each number having {digits} num of bits

    Args:
        a_bin (list[bool]): binary representation of a
        b_bin (list[bool]): binary representation of b
        d_bin (list[bool]): binary representation of d
        digits (int): number of bits of each number 
        name (str): string to identify carry boolean variables

    Returns:
        (Z3-expression): formula representing SAT encoding of binary sum
    """

    #slide SAT-I
    c = [Bool(f"c_{k}_{name}") for k in range(digits+1)]
    c[0] = False
    c[-1] = False

    clauses = []
    for k in range(digits-1,-1,-1):
        # clauses.append((a_bin[k] == b_bin[k]) == (c[k+1] == d_bin[k]))    #TODO: check if it is faster
        clauses.append(d_bin[k] == Or(And(a_bin[k], b_bin[k], c[k+1]), And(a_bin[k], Not(b_bin[k]), Not(c[k+1])), 
                                   And(Not(a_bin[k]), b_bin[k], Not(c[k+1])), And(Not(a_bin[k]), Not(b_bin[k]), c[k+1])))
        clauses.append(c[k] == Or(And(a_bin[k],b_bin[k]), And(a_bin[k],c[k+1]), And(b_bin[k],c[k+1])))

    return And(clauses)

#### Linear Integer constraint over binary numbers

In [14]:
def LinearInteger_bin(x, alpha, beta, name):
    """Encodes into a SAT formula the linear constraint {sum_over_j(alpha[j] | x[j] == True) <= beta}

    Args: 
        x (list[bool]): list of coefficients, i.e. x_j tells wether or not to add alpha_j to the sum
        alpha (list[list[bool]]): list of TODO, each one represented as list[bool] i.e. binary number, whose subset will be summed in the constraint
        beta (list[bool]): binary representing the known term, i.e. the upper bound for the sum
        name (string): to uniquely identify the created variables
    Returns:
        formula (Z3-expression): And of clauses representing SAT encoding of Linear Integer constraint
    """
    n = len(x)
    digits = len(beta)

    # matrix containing temporary results of sum_bin
    d = [[Bool(f"d_{j}_{k}_{name}") for k in range(digits)] for j in range(n)]

    clauses = []

    # row 0
    clauses.append( And(Implies(x[0], equal(d[0], alpha[0], digits)),              # If x[0] == 1 then d_0 == alpha_0
                        Implies(Not(x[0]), all_false(d[0]))))              # elif x[0] == 0 then d_0 == [0..0]
    
    # row j>1
    for j in range(1,n):
        clauses.append( And(Implies(x[j], sum_bin(d[j-1], alpha[j], d[j], digits, f"{name}_{j-1}_{j}")),           # If c_j == 1 then d_j == d_j-1 + alpha_j 
                            Implies(Not(x[j]), equal(d[j], d[j-1], digits))))                                      # elif c_j == 0 then d_j == d_j-1

    # TODO: aggiungere a clause che d_n <= beta
    # I suppose to enter beta as integer (is it the best idea to optimize the program?)
    clauses.append(leq(d[n-1], beta, digits)) 
    # TODO: does it make any sense to check d_j <= beta forall j or better to just do it for d_n?
    # TODO: have to check for overflows in num. digits?

    formula = And(clauses)
    return formula



#### Functions to display solution

In [15]:
def interest_variables(model, a):
    return [[j for j in range(len(a[0])) if model.evaluate(a[i][j])] for i in range(len(a))]

def displayMCP(assignments):
    for i in range(len(assignments)):
        print(f"Courier {i}: items {assignments[i]}")

#### Functions to display matrices of tensor and dist

In [16]:
### Add by Scaio

def tensor(model, bools):
    m = len(bools)
    n = len(bools[0])
    return [[[model.evaluate(bools[i][j][k]) for k in range(n)] for j in range(n)] for i in range(m)]

def display_matrices(tensor, name):
    m = len(tensor)
    n = len(tensor[0])
    for i in range(m):
        print(f'{name}_{i} matrix:')
        for j in range(n):
            print(tensor[i][j])
        print()

def courier_dist(model, r_i, a_i, D, n): # Implemented with the assumption where r_i is the matrix nxn where 
                                         # j is the object and k indicates, with respect to the other columns, the order of delivery
    dist = 0
    actual_pos = n #the origin, it would be n+1
    next_pos = n
    for k in range(n): #sum(a_i)+1):
        for j in range(n):
            if model.evaluate(r_i[k][j]):
                next_pos = j
        dist += D[actual_pos][next_pos]
        actual_pos = next_pos
    dist += D[actual_pos][n] #the origin, it would be n+1
    return dist

def max_dist(model, r, a, D, m, n):
    return max([courier_dist(model, r[i], a[i], D, n) for i in range(m)])

###

#### Function to check that routes are all Hamiltonian cycles

In [64]:
def check_all_hamiltonian(tensor):
    m = len(tensor)
    for i in range(m):
        if not check_hamiltonian(tensor[i]):
            print(f"AAA errore in {i}")
            return False
    return True

def check_hamiltonian(matrix):
    n = len(matrix)
    visited = [0] * n
    v = n-1
    while visited[v] == 0:
        visited[v] = 1
        for k in range(n):
            if matrix[v][k] == True:
                v = k
                break
    num_vertices = sum(row.count(True) for row in matrix)
    return (sum(visited) == num_vertices)


## Assumptions

Assumpions that we can make:
1. $n >= m$
2. The distance matrix $D$ is quasimetric (more info [here](https://proofwiki.org/wiki/Definition:Quasimetric)) => Giving an item to courier who has none is always "same or better" than giving 2 to another courier and none to this one

(Note: 1. + 2. => can add implicit constraint "Every courier has at least one item assigned to it")

## Model 1

In [68]:
# TODO: add (and copy-implement) timeout decorator
def multiple_couriers_planning_1(m, n, l, s, D):
    """Model 1 in Z3 for the Multiple Couriers Planning problem

    Args:
        m (int): number of couriers
        n (int): number of items to deliver
        l (list[int]): l[i] represents the maximum load of courier i, for i = 1..m
        s (list[int]): s[j] represents the size of item j, for j = 1..n
        D (list[list[int]]): (n+1)x(n+1) matrix, with D[i][j] representing the distance from
                             distribution point i to distribution point j
                    
    """
    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars

    # r for routes
    r = [[[Bool(f"r_{j}_{k}_{i}") for k in range(n+1)] for j in range(n+1)] for i in range(m)]  
    # r_kji = 1 indicates that courier i moves from delivery point j to delivery point k in his route
    # n+1 delivery points because considering Origin point as well, representes as n+1-th row and column, for each courier i
    # O(m * n^2) vars

    solver = Solver()

    ## CONSTRAINTS
    # Conversions:
    digits = find_max_digits(s, l)
    s_bin = [int_to_bin(s_j, digits) for s_j in s]
    l_bin = [int_to_bin(l_i, digits) for l_i in l]


    # Constraint 1: every object is assigned to one and only one courier
    for j in range(n):
        solver.add(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))
    
    # Constraint 2: every courier can't exceed its load capacity
    for i in range(m):
        solver.add(LinearInteger_bin(a[i], s_bin, l_bin[i], f"load_capacity_courier_{i}"))

    # Constraint 3: every courier has at least 1 item to deliver (implicit constraint, because n >= m and distance is quasimetric (from discussion forum))
    # TODO: check if it actually speeds up execution
    for i in range(m):
        solver.add(at_least_one_seq(a[i]))

    #TODO: better to use "clauses" list than doing so many solver.add below? Or call entirely a function maybe

    # Constraint 4: routes
    # last row and last column represent the Origin point
    z = [[[Bool(f"reachable_{k}_from_{j}_courier_{i}") for k in range(n+1)] for j in range(n+1)] for i in range(m)] #TODO: move down, just for debugging

    for i in range(m):
        # Constraint 4.1: diagonal is full of zeros, i.e. can't leave from j to go to j
        solver.add(And([Not(r[i][j][j]) for j in range(n+1)]))

        # Constraint 4.2: row j has a 1 iff courier i delivers object j
        # rows
        for j in range(n):
            solver.add(Implies(a[i][j], exactly_one_seq(r[i][j], f"courier_{i}_leaves_{j}")))  # If a_ij then exactly_one(r_ij)
            solver.add(Implies(Not(a[i][j]), all_false(r[i][j])))   # else all_false(r_ij)
        solver.add(exactly_one_seq(r[i][n], f"courier_{i}_leaves_origin"))    # exactly_one in origin point row === courier i leaves from origin

        # Constraint 4.3: column j has a 1 iff courier i delivers object j
        # columns
        for k in range(n):
            solver.add(Implies(a[i][k], exactly_one_seq([r[i][j][k] for j in range(n+1)], f"courier_{i}_reaches_{k}")))  # If a_ij then exactly_one(r_i,:,k)
            solver.add(Implies(Not(a[i][k]), all_false([r[i][j][k] for j in range(n+1)])))   # else all_false(r_i,:,k) 
            # TODO imply false just for the one that weren't alredy counted in the rows (Edo: "Cos?")
        solver.add(exactly_one_seq([r[i][j][n] for j in range(n+1)], f"courier_{i}_returns_to_origin"))         # exactly_one in origin point column === courier i returns to origin
     
        # Constraint 4.4: HAMILTONIAN CYCLE
        # solver.add(Implies(all_false([r[i][j][k] for j in range(n+1)]), Not(z[k])))   #TODO: aggiungere perchè giusto, useless o implicit? Stessa domanda di sotto

        # Edo idea:
        # TODO: is this optimizable?? Look online for encoding of Hamiltonian cycles (n.b. NOT solutions to the general HCP, Hamiltonian Cycle Problem)
        for j in range(n+1):
            for k in range(n+1):
                if k == j:
                    solver.add(Not(z[i][k][k])) # Diagonal has to be zero: we say that cannot reach a node from itself
                # If j->k is in route, then it's also reachable k from j 
                solver.add(Implies(r[i][j][k], z[i][j][k])) # TODO: this is a useless constraint or an implicit (=> useful) one? Check performance without

                # If we can say that k it's reachable from j, it means that either 
                # (i): j->k is in the route
                # or (ii): exists (Or) a node h such that h is reachable from j in route and h->k is in route
                solver.add(Implies(z[i][j][k], Or(r[i][j][k], Or([And(z[i][j][h], r[i][h][k]) for h in range(n+1)]))))  
        # Goal: want that from Origin (row n) we can reach all the delivery points that courier i has to do (i.e. a[i])
        solver.add(equal(a[i], z[i][n][:-1], n))

        # Scaio idea:
        # for k in range(n):
        #     solver.add(Not(all_false([r[i][j][h] for j in range(n, k, -1) for h in range(k+1)])))
        
        

    # Optimization search

    # Binary representation of the distance travelled by courier i
    dist_digits = 41 # TODO
    dist = [[Bool(f"dist_bin_{i}_{k}") for k in range(dist_digits)] for i in range(m)]
    # N.B. could either (1) find an upper bound to the objective value and use it immediately as a constraint (more information immediately, 
    # but is this allowed? can I use any algo to do so?)
    # or (2) run the model with no bound on the objective function the first time, then use the first value found as the initial bound
    # Question: can I do (1) and find a bound that is usually better than the bound found at (2)? If so, this would mean more info + better bound from the start!!

    
    ### TODO: finish implementing this
    # final_model = None
    # while solver.check() == 'sat':
    #     model = solver.model()
    #     obj_value = obj_function(model, D, r)
    #     obj_value_bin = int_to_bin(obj_value)
    #     solver.add(AllLess_bin(r.flatten(), D.flatten(), obj_value_bin)) # TODO: this time, LInearInteger called inside AllLess must be <, not <= !!! Pass it a parameter to use leq or (to implement) lt (less than)
    ### then remove this below 
    print(solver.check())
    final_model = solver.model()
    ### up to this point
    # TODO: re-do thing above again but for BINARY_SEARCH, not LINEAR_SEARCH, then add parameter to the input of the model to choose between which one to use

    if final_model is None:
        return "UNSAT"
    
    ### Add by Scaio

    # print(max_dist(final_model, r, a, D, m, n))
    R = tensor(final_model, r)
    display_matrices(R, 'r')
    print(f"The tensor contains all hamiltonian cycles: {check_all_hamiltonian(R)}")
    
    ###
    return interest_variables(final_model, a)   


### Testing Model 1

In [69]:
m = 1
n = 3
l = [15]
s = [3, 2, 4]
D = [[0, 3, 3, 2],
     [3, 0, 4, 3],
     [3, 4, 0, 3],
     [2, 3, 3, 0]]

displayMCP(multiple_couriers_planning_1(m, n, l, s, D))

sat
r_0 matrix:
[False, True, False, False]
[False, False, False, True]
[True, False, False, False]
[False, False, True, False]

The tensor contains all hamiltonian cycles: True
Courier 0: items [0, 1, 2]


In [70]:
m = 3
n = 7
l = [15, 10, 7]
s = [3, 2, 6, 8, 5, 4, 4]
D = [[0, 3, 3, 6, 5, 6, 6, 2],
     [3, 0, 4, 3, 4, 7, 7, 3],
     [3, 4, 0, 7, 6, 3, 5, 3],
     [6, 3, 7, 0, 3, 6, 6, 4],
     [5, 4, 6, 3, 0, 3, 3, 3],
     [6, 7, 3, 6, 3, 0, 2, 4],
     [6, 7, 5, 6, 3, 2, 0, 4],
     [2, 3, 3, 4, 3, 4, 4, 0]]

displayMCP(multiple_couriers_planning_1(m, n, l, s, D))

sat
r_0 matrix:
[False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False]
[False, False, False, False, True, False, False, False]
[False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, True]
[False, False, False, False, False, False, False, False]
[False, False, True, False, False, False, False, False]
[False, False, False, False, False, False, True, False]

r_1 matrix:
[False, False, False, False, False, False, False, False]
[False, False, False, True, False, False, False, False]
[False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, True]
[False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False]
[False, True, False, False, False, False, False, False]

r_2 matrix:
[False, False, False, False, False, True, False, Fals

In [71]:
m = 1
n = 3
l = [15]
s = [3, 2, 4]
D = [[0, 1, 1, 1],
     [1, 0, 1, 1],
     [1, 1, 0, 1],
     [1, 1, 1, 0]]

displayMCP(multiple_couriers_planning_1(m, n, l, s, D))

sat
r_0 matrix:
[False, True, False, False]
[False, False, False, True]
[True, False, False, False]
[False, False, True, False]

The tensor contains all hamiltonian cycles: True
Courier 0: items [0, 1, 2]


In [53]:
for k in range(3):
    print([(j, h) for j in range(n, k, -1) for h in range(k+1)])

[(3, 0), (2, 0), (1, 0)]
[(3, 0), (3, 1), (2, 0), (2, 1)]
[(3, 0), (3, 1), (3, 2)]
